### Sample code for Comparing NILM algorithms

In [1]:
from __future__ import print_function, division
import time
from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

rcParams['figure.figsize'] = (13, 6)

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.disaggregate import CombinatorialOptimisation, FHMM, MLE

/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/scipy/linalg/__init__.py:191: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/scipy/sparse/lil.py:16: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-p

### Dividing data into train and test set

In [2]:
train = DataSet('/home/shifona/Downloads/mini_project/REDD/redd.h5')
test = DataSet('/home/shifona/Downloads/mini_project/REDD/redd.h5')

/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/tables/__init__.py:90: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .utilsextension import (
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/tables/file.py:35: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import hdf5extension
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/tables/link.py:33: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import linkextension
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/tables/table.py:28: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import tableextension
/home/shifona/anaconda2/envs/nilmtk/lib/python2.7/site-packages/tables/index.py:33: RuntimeWarning: numpy.dtype size changed, may indicate binar

HDF5ExtError: HDF5 error back trace

  File "H5F.c", line 586, in H5Fopen
    unable to open file
  File "H5Fint.c", line 1305, in H5F_open
    unable to lock the file
  File "H5FD.c", line 1839, in H5FD_lock
    driver lock request failed
  File "H5FDsec2.c", line 940, in H5FD_sec2_lock
    unable to lock file, errno = 11, error message = 'Resource temporarily unavailable'

End of HDF5 error back trace

Unable to open/create file '/home/shifona/Downloads/mini_project/REDD/redd.h5'

Let us use building 1 for demo purposes

In [3]:
building = 1

Let's split data at April 30th

In [4]:
train.set_window(end="30-4-2011")
test.set_window(start="30-4-2011")


train_elec = train.buildings[1].elec
test_elec = test.buildings[1].elec

NameError: name 'train' is not defined

### Selecting top-5 appliances

In [5]:
top_5_train_elec = train_elec.submeters().select_top_k(k=5)

NameError: name 'train_elec' is not defined

### Training and disaggregation

In [6]:
def predict(clf, test_elec, sample_period, timezone):
    pred = {}
    gt= {}

    for i, chunk in enumerate(test_elec.mains().load(sample_period=sample_period)):
        chunk_drop_na = chunk.dropna()
        pred[i] = clf.disaggregate_chunk(chunk_drop_na)
        gt[i]={}

        for meter in test_elec.submeters().meters:
            # Only use the meters that we trained on (this saves time!)    
            gt[i][meter] = meter.load(sample_period=sample_period).next()
        gt[i] = pd.DataFrame({k:v.squeeze() for k,v in gt[i].iteritems()}, index=gt[i].values()[0].index).dropna()
        
    # If everything can fit in memory
    gt_overall = pd.concat(gt)
    gt_overall.index = gt_overall.index.droplevel()
    pred_overall = pd.concat(pred)
    pred_overall.index = pred_overall.index.droplevel()

    # Having the same order of columns
    gt_overall = gt_overall[pred_overall.columns]
    
    #Intersection of index
    gt_index_utc = gt_overall.index.tz_convert("UTC")
    pred_index_utc = pred_overall.index.tz_convert("UTC")
    common_index_utc = gt_index_utc.intersection(pred_index_utc)
    
    
    common_index_local = common_index_utc.tz_convert(timezone)
    gt_overall = gt_overall.ix[common_index_local]
    pred_overall = pred_overall.ix[common_index_local]
    appliance_labels = [m.label() for m in gt_overall.columns.values]
    gt_overall.columns = appliance_labels
    pred_overall.columns = appliance_labels
    return gt_overall, pred_overall

In [7]:
classifiers = {'CO':CombinatorialOptimisation(), 'FHMM':FHMM()}
predictions = {}
sample_period = 120
for clf_name, clf in classifiers.iteritems():
    print("*"*20)
    print(clf_name)
    print("*" *20)
    clf.train(top_5_train_elec, sample_period=sample_period)
    gt, predictions[clf_name] = predict(clf, test_elec, 120, train.metadata['timezone'])
   
    

********************
FHMM
********************


NameError: name 'top_5_train_elec' is not defined

In [8]:
def compute_rmse(gt, pred):
    from sklearn.metrics import mean_squared_error
    rms_error = {}
    for appliance in gt.columns:
        rms_error[appliance] = np.sqrt(mean_squared_error(gt[appliance], pred[appliance]))
    return pd.Series(rms_error)
def compute_acc(gt, pred):
    import math
    pd_ = {}
    gt_ = {}
    rerror = {}
    lt = 0
    leng = {}
    napp = []
    for appliance in gt.columns:
        t = len(pred[appliance])
        if lt<t:
            lt = t
            napp = []
        if lt==t:
            napp.append(appliance)
        leng[appliance]= t
        #print(str(appliance) + " : "+str(t))


    for appliance in napp:
        if (leng[appliance] < lt):
            continue
        #print(appliance)
        gt_[appliance] = gt[appliance]
        pd_[appliance] = pred[appliance]
        #print(str(len(pd_[appliance]))+", "+str(len(gt_[appliance])))
    
    #print(napp)
    #print(lt)
    s = 0
    sd = 0
    for i in range(lt):
        for appliance in napp:
            #if isinstance( pd[appliance][0], ( int, long ) ):
            #print( pd_[appliance] )
            #print(i)
            s = s + abs(pd_[appliance][i]-gt_[appliance][i])
            sd = sd + gt_[appliance][i]

    acc_ = 1 - (1.0*s)/sd/2
    #print(acc)
    #print(pd)
    return acc_
    #return pd.Series(rms_error)

In [9]:
rmse = {}
for clf_name in classifiers.keys():
    rmse[clf_name] = compute_rmse(gt, predictions[clf_name])
rmse = pd.DataFrame(rmse)

NameError: name 'gt' is not defined

In [ ]:
rmse

In [ ]:
accu = {}
for clf_name in classifiers.keys():
    accu[clf_name] = compute_acc(gt, predictions[clf_name])
print(accu)

In [ ]:
i=0
for clf_name in classifiers.keys():
    pd=predictions[clf_name]
    for app in gt.columns:
        plt.figure(i)
        i=i+1
        gt[app].head(1000).plot(label="("+clf_name+") Data "+str(app))
        pd[app].head(1000).plot(label="("+clf_name+") Pred "+str(app))
        plt.legend()